<a href="https://www.kaggle.com/code/ahmedanwar89/sales-managers-dataset-eda?scriptVersionId=144180362" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Import Dataset

In [ ]:
df= pd.read_csv('/kaggle/input/sales-manager-dataset/sales_data.csv')

df.head(2)

# Data Cleaning Step

# ***check of data type***

In [ ]:
df.dtypes

In [ ]:
# convert from object to datetime

df['OrderDate']= pd.to_datetime(df['OrderDate'])

# ***check of null values***

In [ ]:
df.isnull().any(), df.isnull().sum()

In [ ]:
df.duplicated().any(), df.duplicated().sum()

In [ ]:
df['Product_Category'].unique()

In [ ]:
df['Product_Code'].unique()

In [ ]:
df[df['Product_Category'].isnull()]['Product_Code']

In [ ]:
# split 'Product_Code' column into ['product_code_letter', 'product_code_number'].
df[['product_code_letter', 'product_code_number']] = df['Product_Code'].str.split(pat= '-', expand= True)

In [ ]:
df.head(2)

In [ ]:
# fill Null values depend on 'product_code_letter' values.
# HC= Healthcare, O= Office, F= Fashion, ENT= Entertainment, H= Home

for x in df.index:
    if df.loc[x, 'product_code_letter'] == 'HC':
        df.loc[x, 'Product_Category']= 'Healthcare'
    
    elif df.loc[x, 'product_code_letter'] == 'O':
        df.loc[x, 'Product_Category']= 'Office'
    
    elif df.loc[x,'product_code_letter'] == 'F':
        df.loc[x, 'Product_Category']= 'Fashion'
    
    elif df.loc[x,'product_code_letter'] == 'ENT':
        df.loc[x, 'Product_Category']= 'Entertainment'
    
    else : df.loc[x, 'Product_Category']= 'Home'

In [ ]:
# check of null values in Product_Category again

df.isna().any(), df.isna().sum()

In [ ]:
df.shape

In [ ]:
# calculate the percetage of null values to all values

(df.isnull().sum() / len(df['Delivery_Time(Days)'])) * 100

In [ ]:
# Null values perentage in 'Delivery_Time(Days)' = 0.51% and it is very small so it is better to drop it.

df.dropna(inplace= True, axis= 0)

In [ ]:
df.isnull().any(), df.isnull().sum()

# ***check duplicated values***

In [ ]:
df.duplicated().any(), df.duplicated().sum()

# ***check of white spaces***

In [ ]:
df.columns

In [ ]:
# strip white spaces in columns title

df.columns.str= df.columns.str.strip()

In [ ]:
# drop columns not need any more

df.drop(columns= ['product_code_letter', 'product_code_number'], axis= 1, inplace= True)

# ***check of outliers***

In [ ]:
df.describe().round(2)

In [ ]:
# choose num values only

df_num = df.select_dtypes(exclude= ['object', 'datetime'])

In [ ]:
df_num.drop(columns= ['OrderID', 'OrderCode'], inplace= True)

In [ ]:
fig, ax = plt.subplots(figsize= (30, 10), nrows= 1, ncols= len(df_num.columns), sharey= False)

labels= df_num.columns

for i in np.arange(0, len(df_num.columns), 1) :
    ax[i].boxplot(df_num.iloc[:, i])
    ax[i].set_xlabel(labels[i], fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
    
plt.show()

# Data Analysis Step

In [ ]:
plt.figure(figsize= (12, 5))
sns.heatmap(data= df.corr(numeric_only= True), annot= True, cmap= 'Blues', vmax= 1, vmin= -1)
plt.title('Relation Between Numric Columns', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.show()

- there are no 'Good Relation' between num columns so, try to find something else to indicate it.

In [ ]:
# calculate profit for each row has Status "Delivered"

for x in df.index:
    if df.loc[x, 'Status'] == 'Delivered':
        df.loc[x, 'profit'] = (df.loc[x, 'Quantity'] * df.loc[x, 'UnitPrice(USD)']) - df.loc[x, 'Shipping_Cost(USD)']
    else: df.loc[x, 'profit'] = 0

In [ ]:
# select num data

num_data = df.select_dtypes(exclude= ['object', 'datetime']).drop(columns= ['OrderID', 'OrderCode'])

In [ ]:
plt.figure(figsize= (12, 5))
sns.heatmap(data= num_data.corr(numeric_only= True), annot= True, cmap= 'Blues', vmax= 1, vmin= -1)
plt.title('Relationship Coeff. Between Numric Columns', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.show()

# **Q: How much profit for each Product_Category that has Delivered Status?**

In [ ]:
# select data

df[df['Status'] == 'Delivered'].groupby('Product_Category').agg({'profit': 'sum'})

In [ ]:
df[df['Status'] == 'Delivered'].groupby('Product_Category').agg({'profit': 'sum'}).plot(kind= 'barh', grid= True, color= '#00004f')
plt.title('Sum of profit for each product category', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.xlabel('sum of profit', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.ylabel('product category', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.show()

In [ ]:
df['Status'].value_counts().sort_values(ascending= False)

- Alot of orders didn't deliverd, avoid that by filtring orders which have status 'Deliverd' and have profit.

In [ ]:
# filter data with profit column

df[df['profit'] != 0].head()

In [ ]:
plt.figure(figsize= (12, 4))
sns.heatmap(data= df[df['profit'] != 0].corr(numeric_only= True),cmap='Blues', annot=True, vmax= 1,vmin= -1)
plt.title('Relationship Coeff. Between Numric Columns', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.show()

- **Very Good Relation**  
- Quantity & profit 0.89.

In [ ]:
plt.figure(figsize= (15, 5))
sns.regplot(data= df[df['profit'] != 0], x= 'Quantity', y= 'profit')
plt.title('Relationship Between Quantity & Profit', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.xlabel('Quantity', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.ylabel('Profit', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.show()

# **Q: How many Quantity in orders for each Product Category for Deliverd Orders?**

In [ ]:
# select data

df[df['profit'] != 0][['Product_Category', 'Quantity']].groupby('Product_Category').sum().sort_values(by= 'Quantity', ascending= False)

# **Q: How many Quantity of Orders for each Shipping Address for Deliverd Orders?**

In [ ]:
df[df['profit'] != 0][['Shipping_Address', 'Quantity']].groupby('Shipping_Address').sum().sort_values(by= 'Quantity', ascending= False)

# **Q: How much profit for each sales manager regarding to quantity?**

In [ ]:
df[df['profit']!=0][['Sales_Manager', 'Quantity', 'profit']].groupby('Sales_Manager').sum().sort_values(by='profit',ascending=False)

In [ ]:
manager_quantity_profit = df[df['profit'] != 0].groupby('Sales_Manager').agg({'Quantity': 'sum', 'profit': 'sum'})
manager_quantity_profit

In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols= 2, figsize= (20, 5))

ax[0].bar(x= manager_quantity_profit.index, height= manager_quantity_profit['Quantity'], color= '#00004f')
ax[0].set_ylabel('Sum of Quantity', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
ax[0].set_title('Sum of Quantity for each Sales Manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})

ax[1].bar(x= manager_quantity_profit.index, height= manager_quantity_profit['profit'], color= '#b60000')
ax[1].set_ylabel('Sum of Profit', fontdict= {'size': 12, 'weight': 'bold', 'color': '#b60000'})
ax[1].set_title('Sum of Profit for each Sales Manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#b60000'})

plt.show()

- orders with Status = 'Shipped' are consider as waiting profit, let's try analyzing it

# **Q: How much waiting profit for orders have status 'Shipped' for each product category?**

In [ ]:
df['Status'].unique()

In [ ]:
# calculate new column call waiting profit

for x in df.index:
    if df.loc[x, 'Status'] == 'Shipped':
        df.loc[x, 'waiting_profit'] = (df.loc[x, 'Quantity'] * df.loc[x, 'UnitPrice(USD)']) - df.loc[x, 'Shipping_Cost(USD)']
    else: df.loc[x, 'waiting_profit'] = 0

In [ ]:
df['waiting_profit'].isnull().any(), df['waiting_profit'].isnull().sum()

In [ ]:
plt.figure(figsize= (15, 5))
sns.heatmap(data= df[df['Status']=='Shipped'].corr(numeric_only=True),cmap='Blues',annot=True, vmax= 1, vmin= -1)
plt.title('Relationship Coeff. Between Waiting Profit & Other Numric Feilds', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.show()

In [ ]:
df[df['Status'] == 'Shipped'].groupby('Product_Category').agg({'waiting_profit': 'sum'}).sort_values(by='waiting_profit',ascending=False)

# **Q: How many Quantity for each Product Category in Shipped Orders Status?**

In [ ]:
df[df['Status']=='Shipped'].groupby('Product_Category').agg({'Quantity': 'sum'}).sort_values(by='Quantity',ascending=False)

# **Q: How much waiting profit for each sales manager regarding to Quantity?**

In [ ]:
# select data

manager_quantity_w8_profit = \
df[df['waiting_profit'] != 0].groupby('Sales_Manager').agg({'waiting_profit': 'sum', 'Quantity': 'sum'}).sort_values(by='waiting_profit',ascending= False)

manager_quantity_w8_profit

In [ ]:
fig, ax = plt.subplots(nrows= 1, ncols= 2, figsize= (20, 5), sharey= False)

ax[0].bar(x= manager_quantity_w8_profit.index, height= manager_quantity_w8_profit['waiting_profit'], color= '#00004f')
ax[0].set_title('sum of waiting profit for each sales manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})

ax[1].bar(x= manager_quantity_w8_profit.index, height= manager_quantity_w8_profit['Quantity'], color= '#b60000')
ax[1].set_title('sum of quantity for each sales manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#b60000'})

plt.show()

# **Q: How much waiting profit and profit for each sales manager, and Whoes the big expected profit?**

In [ ]:
# select data

df.groupby('Sales_Manager').agg({'profit': 'sum', 'waiting_profit': 'sum'})

In [ ]:
data = df.groupby('Sales_Manager').agg({'profit': 'sum', 'waiting_profit': 'sum'})

fig, ax = plt.subplots(nrows= 1, ncols= 2, figsize= (20, 5), sharey= False)

ax[0].bar(x= data.index, height= data['profit'], color= '#00004f')
ax[0].set_title('sum of profit for each sales manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})

ax[1].bar(x= data.index, height= data['waiting_profit'], color= '#b60000')
ax[1].set_title('sum of waiting profit for each sales manager', fontdict= {'size': 16, 'weight': 'bold', 'color': '#b60000'})

plt.show()

# **Q: what is the percentage of Not Deliverd orders to Deliverd orders?**

In [ ]:
df['Status'].value_counts()

In [ ]:
2558 / (2435 + 2558) * 100

# **Q: How much profit by Date?**

In [ ]:
df[df['profit'] != 0].groupby('OrderDate').agg({'profit': 'sum'})

In [ ]:
plt.figure(figsize= (20, 5))
plt.plot(df[df['profit'] != 0].groupby('OrderDate').agg({'profit': 'sum'}), color= '#00004f', marker= '.')
plt.title('Sum of Profit by Date', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
plt.xlabel('Date', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.ylabel('Sum of Profit', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
plt.show()

# **Q: How many Quantity by Date?**

In [ ]:
df[df['profit'] != 0].groupby('OrderDate').agg({'Quantity': 'sum'}).sort_values(by='OrderDate')

In [ ]:
plt.figure(figsize= (20, 5))
plt.plot(df[df['profit'] != 0].groupby('OrderDate').agg({'Quantity': 'sum'}).sort_values(by='OrderDate'), color= '#b60000', marker= '.')
plt.title('Sum of Quantity by Date', fontdict= {'size': 16, 'weight': 'bold', 'color': '#b60000'})
plt.xlabel('Date', fontdict= {'size': 12, 'weight': 'bold', 'color': '#b60000'})
plt.ylabel('Sum of Quantity', fontdict= {'size': 12, 'weight': 'bold', 'color': '#b60000'})
plt.show()

- all orders were in 2021, try to analyzing by month

# **Q: How much profit by month regarding to Quantity in 2021?**

In [ ]:
df['month']= pd.DatetimeIndex(df['OrderDate']).month

In [ ]:
df[df['profit']!=0].groupby('month').agg({'profit': 'sum', 'Quantity': 'sum'}).sort_values(by='month')

In [ ]:
data = df[df['profit']!=0].groupby('month').agg({'profit': 'sum', 'Quantity': 'sum'}).sort_values(by='month')

fig, ax = plt.subplots(nrows= 1, ncols= 2, figsize= (20, 5))

ax[0].plot(data.index, data['profit'], color= '#00004f', marker= 'o')
ax[0].set_title('Sum of Profit by Month', fontdict= {'size': 16, 'weight': 'bold', 'color': '#00004f'})
ax[0].set_xlabel('Month', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})
ax[0].set_ylabel('Sum of Profit', fontdict= {'size': 12, 'weight': 'bold', 'color': '#00004f'})

ax[1].plot(data.index, data['Quantity'], color= '#b60000', marker= 'D')
ax[1].set_title('Sum of Quantity by Month', fontdict= {'size': 16, 'weight': 'bold', 'color': '#b60000'})
ax[1].set_xlabel('Month', fontdict= {'size': 12, 'weight': 'bold', 'color': '#b60000'})
ax[1].set_ylabel('Sum of Quantity', fontdict= {'size': 12, 'weight': 'bold', 'color': '#b60000'})

plt.show()

In [ ]:
df.groupby(['OrderDate','Product_Category']).agg({'profit': 'sum'}).sort_values(by= 'OrderDate')

In [ ]:
df.pivot_table(index='OrderDate',columns='Product_Category',values='profit',aggfunc='sum')